## Changelog

#### 17/01/25

- Found the dataset that I want to use from this link (https://www.geeksforgeeks.org/dataset-for-sentiment-analysis/#1-imdb-reviews-dataset)
- This is a sentiment analysis dataset taken from IMDB movie reviews and it is used to find out whether or not a review is positive or negative based on the words used in the reviews
- It has 50,000 rows, this isn't the biggest dataset but it is a good starting point whilst getting used to the new model and I want to see how accurate I can get it at predicting sentiments
- A multinomial Bayesian Classifier seems to be the best fit for this dataset as this will have a high number of features which multinomial bayesian classifiers are good at handling.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ds = pd.read_csv('IMDB-Dataset.csv')
print(ds.head())
# This isjust importing the data and printing the first 5 rows of the data to ensure that the data is imported correctly

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
categories = ['Positive', 'Negative']

from sklearn.model_selection import train_test_split
